In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./datasets/intermediate/shuffled_44k.csv')
df

,URL,URL_Length,Url_Shortening,Global_Ranking,Has_Suspicious_Params,Num_Languages,Uses_HTTPS,Has_User_Info,User_Info_Length,Num_Subdomains,...,Num_Dash_Query,Num_AtSymbol_Query,Num_TildeSymbol_Query,Num_Underscore_Query,Num_Percent_Query,Num_Ampersand_Query,Num_Hash_Query,Hex_Encoded_Characters_Query,Has_Fragment,label
0,my.gov.gr,9,0,806817,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,http://sleepmaskz.com/wp-admin/files-logs/logs...,102,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,ec.or.ug,8,0,769366,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,orbitpages.online,17,0,-1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,dstv.co.za,10,0,510807,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43993,liquorapps.com,14,0,-1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43994,https://ezdrive.com-ejcl.xin/,29,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
43995,https://docs.google.com/presentation/d/e/2PACX...,175,0,1,0,1,1,0,0,0,...,0,0,0,0,0,2,0,0,0,1
43996,https://sites.google.com/view/bienvenidobanrural,48,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [3]:
oldColumns = df.columns
df.sample(frac=1).reset_index(drop=True)

if 'Ssl_Info' in df.columns: 
    df.drop(['Ssl_Info'], axis=1, inplace=True)
    print('Ssl_Info dropped')
if 'Unnamed: 0' in df.columns:
    df.drop(['Unnamed: 0'], axis=1, inplace=True)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

nonNumeric = df.select_dtypes(include=['object']).columns
boolCols = df.select_dtypes(include=['bool']).columns
df[boolCols] = df[boolCols].astype(int)

print(nonNumeric)
print(boolCols)

for col in nonNumeric:
    if col != 'URL':
        df[col] = le.fit_transform(df[col])
        print(f'{col} encoded')




Index(['URL'], dtype='object')
Index([], dtype='object')


In [4]:
df_nourl = df.drop(columns=["URL"])

correlation_with_label = df_nourl.corr(numeric_only=True)["label"].sort_values(ascending=False)

high_corr = correlation_with_label[abs(correlation_with_label) > 0.7]

feature_variance = df_nourl.drop(columns=["label"]).var()
unique_counts = df_nourl.drop(columns=["label"]).nunique()

high_corr, feature_variance.sort_values(), unique_counts.sort_values()

(label            1.000000
 Uses_HTTPS       0.952354
 Num_Dots_Path   -0.810707
 Name: label, dtype: float64,
 Num_TildeSymbol_Subdomain           0.000000e+00
 Num_AtSymbol_Subdomain              0.000000e+00
 Num_Underscore_Domain               0.000000e+00
 Num_TildeSymbol_Domain              0.000000e+00
 Hex_Encoded_Characters_Subdomain    0.000000e+00
                                         ...     
 Length_Query                        1.172422e+03
 URL_Length                          1.719405e+04
 TLD                                 5.609838e+04
 Global_Ranking                      2.947197e+11
 Domain_Age                                   NaN
 Length: 65, dtype: float64,
 Domain_Age                       0
 Num_TildeSymbol_Subdomain        1
 Num_TildeSymbol_Domain           1
 Num_AtSymbol_Subdomain           1
 Num_Ampersand_Subdomain          1
                              ...  
 Length_Path                    145
 Length_Query                   250
 URL_Length           

In [5]:
# Find columns with 0 variance
zero_variance = feature_variance[feature_variance == 0]
zero_variance



Num_AtSymbol_Subdomain              0.0
Num_TildeSymbol_Subdomain           0.0
Num_Percent_Subdomain               0.0
Num_Ampersand_Subdomain             0.0
Num_Hash_Subdomain                  0.0
Hex_Encoded_Characters_Subdomain    0.0
Num_Dash_Domain                     0.0
Num_AtSymbol_Domain                 0.0
Num_TildeSymbol_Domain              0.0
Num_Underscore_Domain               0.0
Num_Percent_Domain                  0.0
Num_Ampersand_Domain                0.0
Num_Hash_Domain                     0.0
Hex_Encoded_Characters_Domain       0.0
Num_Hash_Path                       0.0
Num_Hash_Query                      0.0
dtype: float64

In [6]:
df_nourl = df_nourl.drop(columns=zero_variance.index)


In [7]:
df_nourl

,URL_Length,Url_Shortening,Global_Ranking,Has_Suspicious_Params,Num_Languages,Uses_HTTPS,Has_User_Info,User_Info_Length,Num_Subdomains,Num_Digits_Subdomain,...,Num_Dots_Query,Num_Dash_Query,Num_AtSymbol_Query,Num_TildeSymbol_Query,Num_Underscore_Query,Num_Percent_Query,Num_Ampersand_Query,Hex_Encoded_Characters_Query,Has_Fragment,label
0,9,0,806817,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,102,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,8,0,769366,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,17,0,-1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10,0,510807,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43993,14,0,-1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43994,29,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
43995,175,0,1,0,1,1,0,0,0,0,...,0,0,0,0,0,0,2,0,0,1
43996,48,0,1,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [8]:
high_corr = correlation_with_label[abs(correlation_with_label) > 0.7]

high_corr.sort_values(ascending=False)

label            1.000000
Uses_HTTPS       0.952354
Num_Dots_Path   -0.810707
Name: label, dtype: float64

In [9]:
df_nourl.drop(['Uses_HTTPS', 'Num_Dots_Path'], axis=1, inplace=True)

df_nourl

unique_counts = df_nourl.nunique()



In [10]:
unique_counts.sort_values(ascending=False)


Global_Ranking                  18528
TLD                               763
URL_Length                        353
Length_Query                      250
Length_Path                       145
Num_Digits_Query                  120
Num_Digits_Path                    69
Length_Subdomain                   44
Length_Domain                      42
Num_Percent_Query                  24
Hex_Encoded_Characters_Query       24
Num_Digits_Subdomain               23
Num_Dash_Query                     23
Num_Underscore_Query               23
Num_Hyphens_Query                  21
Num_Query_Params                   19
Num_Ampersand_Query                17
Num_Digits_Domain                  16
Num_Dots_Query                     15
User_Info_Length                   13
Num_Hyphens_Path                   13
Path_Level                         13
Num_Dash_Path                      12
Num_Underscore_Path                12
Hex_Encoded_Characters_Path        10
Num_Percent_Path                   10
Num_Hyphens_

In [11]:
nan_counts = df_nourl.isna().sum()
nan_counts.sort_values(ascending=False)

df_nourl.drop(['Domain_Age'], axis=1, inplace=True)


In [12]:
df_nourl


,URL_Length,Url_Shortening,Global_Ranking,Has_Suspicious_Params,Num_Languages,Has_User_Info,User_Info_Length,Num_Subdomains,Num_Digits_Subdomain,Num_Hyphens_Subdomain,...,Num_Dots_Query,Num_Dash_Query,Num_AtSymbol_Query,Num_TildeSymbol_Query,Num_Underscore_Query,Num_Percent_Query,Num_Ampersand_Query,Hex_Encoded_Characters_Query,Has_Fragment,label
0,9,0,806817,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,102,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,8,0,769366,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,17,0,-1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10,0,510807,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43993,14,0,-1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43994,29,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
43995,175,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,1
43996,48,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [13]:
low_variance = feature_variance[feature_variance < 0.01]
low_variance


Has_Suspicious_Params               0.003984
Has_User_Info                       0.000273
Num_Dots_Subdomain                  0.007494
Num_Dash_Subdomain                  0.000091
Num_AtSymbol_Subdomain              0.000000
Num_TildeSymbol_Subdomain           0.000000
Num_Underscore_Subdomain            0.000091
Num_Percent_Subdomain               0.000000
Num_Ampersand_Subdomain             0.000000
Num_Hash_Subdomain                  0.000000
Hex_Encoded_Characters_Subdomain    0.000000
Has_Ip                              0.004885
Num_Dash_Domain                     0.000000
Num_AtSymbol_Domain                 0.000000
Num_TildeSymbol_Domain              0.000000
Num_Underscore_Domain               0.000000
Num_Percent_Domain                  0.000000
Num_Ampersand_Domain                0.000000
Num_Hash_Domain                     0.000000
Hex_Encoded_Characters_Domain       0.000000
Has_Port                            0.000205
Num_AtSymbol_Path                   0.000591
Num_TildeS

In [14]:
for col in low_variance.index:
    if col in df_nourl.columns:
        df_nourl.drop(col, axis=1, inplace=True)

df_nourl



,URL_Length,Url_Shortening,Global_Ranking,Num_Languages,User_Info_Length,Num_Subdomains,Num_Digits_Subdomain,Num_Hyphens_Subdomain,Length_Subdomain,Num_Digits_Domain,...,Num_Hyphens_Query,Length_Query,Num_Query_Params,Num_Dots_Query,Num_Dash_Query,Num_Underscore_Query,Num_Percent_Query,Num_Ampersand_Query,Hex_Encoded_Characters_Query,label
0,9,0,806817,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,102,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,8,0,769366,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,17,0,-1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10,0,510807,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43993,14,0,-1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43994,29,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
43995,175,0,1,1,0,0,0,0,0,0,...,0,43,3,0,0,0,0,2,0,1
43996,48,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [15]:
# df_nourl.to_csv('./newDatasets/clean_44k.csv', index=False)
low_var = df_nourl.loc[:, df_nourl.nunique() <= 2].columns
print("Low variance features:", low_var.tolist())
correlations = df_nourl.corr(numeric_only=True)["label"].sort_values(ascending=False)
print('Correlations:', correlations)

df_nourl.drop(['Global_Ranking','Path_Level'], axis=1, inplace=True)

Low variance features: ['Url_Shortening', 'Num_Dots_Domain', 'label']
Correlations: label                           1.000000
Path_Level                      0.654791
Url_Shortening                  0.259904
Num_Digits_Path                 0.220106
Num_Query_Params                0.208771
Num_Ampersand_Query             0.171634
URL_Length                      0.159037
Num_Dash_Path                   0.157366
Num_Underscore_Path             0.157366
Length_Query                    0.135777
Num_Subdomains                  0.127560
Num_Hyphens_Domain              0.116264
Num_Hyphens_Path                0.108148
Num_Digits_Query                0.099472
TLD                             0.094314
Num_Dots_Query                  0.089101
Length_Subdomain                0.077282
Num_Dots_Domain                 0.070239
Length_Path                     0.056906
Num_Digits_Subdomain            0.056721
Num_Underscore_Query            0.055077
Num_Dash_Query                  0.055077
Num_Hyphens_Qu

In [17]:
df_nourl.to_csv('./datasets/clean_44k.csv', index=False)
df_nourl

,URL_Length,Url_Shortening,Num_Languages,User_Info_Length,Num_Subdomains,Num_Digits_Subdomain,Num_Hyphens_Subdomain,Length_Subdomain,Num_Digits_Domain,Num_Hyphens_Domain,...,Num_Hyphens_Query,Length_Query,Num_Query_Params,Num_Dots_Query,Num_Dash_Query,Num_Underscore_Query,Num_Percent_Query,Num_Ampersand_Query,Hex_Encoded_Characters_Query,label
0,9,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,102,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,8,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,17,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43993,14,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43994,29,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
43995,175,0,1,0,0,0,0,0,0,0,...,0,43,3,0,0,0,0,2,0,1
43996,48,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [18]:
removed_columns = set(oldColumns) - set(df_nourl.columns)
print("Columns removed from oldColumns to new columns:", removed_columns, '\n', 'Total columns removed:', len(removed_columns))

Columns removed from oldColumns to new columns: {'Hex_Encoded_Characters_Domain', 'Has_Suspicious_Params', 'Has_User_Info', 'Num_AtSymbol_Path', 'Num_Dots_Path', 'Num_Ampersand_Path', 'Num_Underscore_Subdomain', 'Num_Ampersand_Domain', 'Uses_HTTPS', 'Num_TildeSymbol_Subdomain', 'Num_Dash_Subdomain', 'Num_DoubleSlash_Path', 'Global_Ranking', 'Num_Dots_Subdomain', 'Num_Dash_Domain', 'Num_Hash_Query', 'Num_TildeSymbol_Path', 'Has_Ip', 'Num_Underscore_Domain', 'Num_Ampersand_Subdomain', 'Num_TildeSymbol_Query', 'URL', 'Num_Percent_Subdomain', 'Hex_Encoded_Characters_Subdomain', 'Path_Level', 'Num_AtSymbol_Domain', 'Num_AtSymbol_Subdomain', 'Num_Percent_Domain', 'Has_Fragment', 'Num_Hash_Domain', 'Has_Port', 'Num_Hash_Subdomain', 'Num_AtSymbol_Query', 'Num_Hash_Path', 'Domain_Age', 'Num_TildeSymbol_Domain'} 
 Total columns removed: 36


In [19]:
removed_columns = ['Num_Dots_Path','Ssl_Info', 'Num_Dash_Subdomain', 'Path_Level', 'Has_Suspicious_Params', 'Num_Hash_Domain', 'Num_AtSymbol_Subdomain', 'Num_TildeSymbol_Domain', 'Num_Underscore_Subdomain', 'Num_AtSymbol_Path', 'Num_Hash_Path', 'Num_Ampersand_Subdomain', 'Num_DoubleSlash_Path', 'Has_Fragment', 'Num_TildeSymbol_Query', 'URL', 'Num_Hash_Query', 'Hex_Encoded_Characters_Subdomain', 'Has_Port', 'Num_Dots_Subdomain', 'Uses_HTTPS', 'Has_User_Info', 'Global_Ranking', 'Num_Dash_Domain', 'Num_AtSymbol_Domain', 'Domain_Age', 'Num_Ampersand_Domain', 'Num_TildeSymbol_Subdomain', 'Num_Percent_Subdomain', 'Num_Percent_Domain', 'Hex_Encoded_Characters_Domain', 'Has_Ip', 'Num_Underscore_Domain', 'Num_Hash_Subdomain', 'Num_AtSymbol_Query', 'Num_Ampersand_Path', 'Num_TildeSymbol_Path']

val_df = pd.read_csv('./datasets/validation_ds.csv')
val_df.drop(columns=removed_columns, inplace=True)

val_df.sample(frac=1).reset_index(drop=True)

if 'Ssl_Info' in val_df.columns: 
    val_df.drop(['Ssl_Info'], axis=1, inplace=True)
    print('Ssl_Info dropped')
if 'Unnamed: 0' in val_df.columns:
    val_df.drop(['Unnamed: 0'], axis=1, inplace=True)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

nonNumeric = val_df.select_dtypes(include=['object']).columns
boolCols = val_df.select_dtypes(include=['bool']).columns
val_df[boolCols] = val_df[boolCols].astype(int)

print(nonNumeric)
print(boolCols)

for col in nonNumeric:
    if col != 'URL':
        val_df[col] = le.fit_transform(val_df[col])
        print(f'{col} encoded')



Index(['TLD'], dtype='object')
Index(['Url_Shortening'], dtype='object')
TLD encoded


In [20]:
val_df.to_csv('./datasets/cleaned/clean_val.csv', index=False)
val_df

,URL_Length,Url_Shortening,Num_Languages,User_Info_Length,Num_Subdomains,Num_Digits_Subdomain,Num_Hyphens_Subdomain,Length_Subdomain,Num_Digits_Domain,Num_Hyphens_Domain,...,Num_Hyphens_Query,Length_Query,Num_Query_Params,Num_Dots_Query,Num_Dash_Query,Num_Underscore_Query,Num_Percent_Query,Num_Ampersand_Query,Hex_Encoded_Characters_Query,label
0,175,0,1,0,0,0,0,0,0,0,...,0,43,3,0,0,0,0,2,0,1
1,67,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,23,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,40,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,175,0,1,0,0,0,0,0,0,0,...,0,43,3,0,0,0,0,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,10,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
396,13,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
397,7,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
398,12,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
